In [1]:
# Necessary libraries
import pandas as pd
import sqlite3
import datetime


In [2]:
# Connect to the SQLite database file
conn = sqlite3.connect(r'C:\Users\klaud\Desktop\Code\Data\Markforged_Data.db')
c = conn.cursor()

In [3]:
# Function that fetches all sqlite tables
def sql_fetch(conn):

    c.execute('SELECT name from sqlite_master where type= "table"')

    print(c.fetchall())

sql_fetch(conn)

[('M_1',), ('Delays_1',), ('M_2',), ('Delays_2',), ('M_3',), ('Delays_3',), ('M_4',), ('Delays_4',), ('M_5',), ('Delays_5',), ('M_6',), ('Delays_6',), ('M_7',), ('Delays_7',), ('M_8',), ('Delays_8',), ('M_9',), ('Delays_9',)]


# Selecting first dataset and cleaning it

#### M_1, M_4, and M_9 represents the three prints that succeded without crashing the chromedriver

In [4]:
#Selecting the dataset/table
#From first Print
df1 = pd.read_sql_query("SELECT * FROM M_1", conn)
df1_delays = pd.read_sql_query("SELECT * FROM Delays_1", conn)
#Merging delays and print data, without selecting date as it will be duplicate
df1 = pd.concat([df1, df1_delays[['timetime','timeperf_counter']]], axis=1)


In [5]:
#Inspecting the dataset 
df1

,Plastic_Temperature,Fiber_Temperature,Machine_state,Date,Current_Layer,Total_Layers,Printed_Object,User,Time_left,Plastic_material,timetime,timeperf_counter
0,36,39,Printing,2021-05-04 11:29:16,0,105,Leirmo_Exp1_Main_Artifact,Klaudijus Natys,3h 21,Onyx,0.207165,0.207148
1,36,39,Printing,2021-05-04 11:29:17,0,105,Leirmo_Exp1_Main_Artifact,Klaudijus Natys,3h 21,Onyx,0.166713,0.166372
2,279,228,Printing,2021-05-04 11:32:01,0,105,Leirmo_Exp1_Main_Artifact,Klaudijus Natys,3h 21,Onyx,0.258708,0.259024
3,279,228,Printing,2021-05-04 11:32:02,0,105,Leirmo_Exp1_Main_Artifact,Klaudijus Natys,3h 21,Onyx,0.128497,0.128503
4,279,228,Printing,2021-05-04 11:32:04,0,105,Leirmo_Exp1_Main_Artifact,Klaudijus Natys,3h 21,Onyx,0.115045,0.115197
...,...,...,...,...,...,...,...,...,...,...,...,...
10423,274,229,Printing,2021-05-04 14:55:26,105,105,Leirmo_Exp1_Main_Artifact,Klaudijus Natys,1,Onyx,0.138683,0.138682
10424,274,229,Printing,2021-05-04 14:55:27,105,105,Leirmo_Exp1_Main_Artifact,Klaudijus Natys,1,Onyx,0.117360,0.117687
10425,274,229,Printing,2021-05-04 14:55:28,105,105,Leirmo_Exp1_Main_Artifact,Klaudijus Natys,1,Onyx,0.131168,0.131102
10426,274,229,Printing,2021-05-04 14:55:29,105,105,Leirmo_Exp1_Main_Artifact,Klaudijus Natys,1,Onyx,0.120317,0.120336


## Identifying the process start and end (removing data that are not the actual process)

In [6]:
#From data we can see that there are some false positives.
#First rows show hotend temperature is 36 although "Machine_state" is Printing it stands.
#Looking at first rows of "Current_layer", layer 0 is indicated, which indicates that the printing process has not started.
#Convertint values of interest into integers
df1 = df1.astype({"Current_Layer": int, 'Plastic_Temperature': int})

In [7]:
#Selecting the data only from first layer  
df_filtered1 = df1[df1['Current_Layer'] > 0]
df_filtered1 = df_filtered1.iloc[:len(df_filtered1) - (df_filtered1.Current_Layer.to_numpy() == int(df1['Total_Layers'].unique()))[::-1].argmax()]


### Selecting Data from second and third print and performing the same steps. However, each dataset was looked through separately, in order to confirm that identifying of process start and end is the same for all datasets

In [8]:

# Data From second Print
df2 = pd.read_sql_query("SELECT * FROM M_4", conn)
df2_delays = pd.read_sql_query("SELECT * FROM Delays_4", conn)
df2 = pd.concat([df2, df2_delays[['timetime','timeperf_counter']]], axis=1)
df2 = df2.astype({"Current_Layer": int, 'Plastic_Temperature': int})
df_filtered2 = df2[df2['Current_Layer'] > 0]

#Additioanlly removing all data after last occurance of final layer
#This is because there are more data in this table, which represents other process
df_filtered2 = df_filtered2.iloc[:len(df_filtered2) - (df_filtered2.Current_Layer.to_numpy() == int(df2['Total_Layers'].unique()))[::-1].argmax()]

In [9]:
# Data From third Print
df3 = pd.read_sql_query("SELECT * FROM M_9", conn)
df3_delays = pd.read_sql_query("SELECT * FROM Delays_9", conn)
df3 = pd.concat([df3, df3_delays[['timetime','timeperf_counter']]], axis=1)

df3 = df3.astype({"Current_Layer": int, 'Plastic_Temperature': int})
df_filtered3 = df3[df3['Current_Layer'] > 0]

# Merging datasets from each print

In [10]:
#Adding column which identifies part number
df_filtered1 = df_filtered1.assign(Part = 1)
df_filtered2 = df_filtered2.assign(Part = 2)
df_filtered3 = df_filtered3.assign(Part = 3)

data_frames = [df_filtered1, df_filtered2, df_filtered3]
#Merge data frames and generate new indexes 
df = pd.concat(data_frames,ignore_index=True)


In [11]:
#Checking for missing values "NaN"
df.isna().any()

Plastic_Temperature    False
Fiber_Temperature      False
Machine_state          False
Date                   False
Current_Layer          False
Total_Layers           False
Printed_Object         False
User                   False
Time_left              False
Plastic_material       False
timetime               False
timeperf_counter       False
Part                   False
dtype: bool

In [12]:
#Checking if all data variables contain same amount of data
df.count()

Plastic_Temperature    31074
Fiber_Temperature      31074
Machine_state          31074
Date                   31074
Current_Layer          31074
Total_Layers           31074
Printed_Object         31074
User                   31074
Time_left              31074
Plastic_material       31074
timetime               31074
timeperf_counter       31074
Part                   31074
dtype: int64

In [13]:
#Converting Time_left from "3h 21" to "201" in form of minutes
df['Time_left'] = df['Time_left'].astype(str) + 'minutes'
df['Time_left'] = df['Time_left'].str.replace('h',' hours')
df['Time_left'] = pd.to_timedelta(df['Time_left']).dt.total_seconds()/60

In [14]:
#Converting date to datetime format
df.loc[:,'Date'] = pd.to_datetime(df['Date'])
#Assuring that the data is sorted in asceding order
df = df.sort_values(by='Date',ascending=True)

In [15]:
#Static values should contain only one unique value, therefore investigating if that is the case here
for k, v in df[['Machine_state',
                 'Total_Layers',
                 'Printed_Object',
                 'User',
                 'Plastic_material']].nunique().to_dict().items():
    print('{} unique values : {}'.format(k,v))

Machine_state unique values : 1
Total_Layers unique values : 1
Printed_Object unique values : 1
User unique values : 1
Plastic_material unique values : 1


In [16]:
#Renaming so all datasets contain same names of same data types
df = df.rename(columns={'Plastic_Temperature':'Hot_end_temperature','Current_Layer':'Current_layer'})
#Same step is performed for data of part 1-3 
df_filtered1 = df_filtered1.rename(columns={'Plastic_Temperature':'Hot_end_temperature','Current_Layer':'Current_layer'})
df_filtered2 = df_filtered2.rename(columns={'Plastic_Temperature':'Hot_end_temperature','Current_Layer':'Current_layer'})
df_filtered3 = df_filtered3.rename(columns={'Plastic_Temperature':'Hot_end_temperature','Current_Layer':'Current_layer'})
#Dropping Fiber_temperature,as it was not printed with fiber hotend 
df.drop('Fiber_Temperature', axis='columns', inplace=True)


In [94]:
#Storing dataframes as csv files where df1-df3 represents print1-print3 and df represents all print data
df1 = df[df['Part'] == 1]
df2 = df[df['Part'] == 2]
df3 = df[df['Part'] == 3]
df.to_csv("df.csv",   index=False)
df1.to_csv("df1.csv", index=False)
df2.to_csv("df2.csv", index=False)
df3.to_csv("df3.csv", index=False)
